In [ ]:
import pandas as pd
from functools import reduce
import operator 

In [ ]:
fileName = 'data.json'
df = pd.read_json(fileName, lines=True, dtype={'fullVisitorId': pd.Int64Dtype})

# Verifica se uma chave está presente em um campo
def check(attr, field):
    return field in attr

df.head(3)

### Criacao do id unico

In [ ]:
# Criacao do id unico
df['id'] = df.apply (lambda row: '{}-{}'.format(row['fullVisitorId'],row['visitId']), axis=1)

df[['fullVisitorId','visitId','id']]

### Contagem de pageviews

In [ ]:
data = df['totals']

def total(line): 
    views = 0
    if ( 'pageviews' in line ):
        views = line['pageviews']

    return int(views)
    
sumPageViews = reduce( operator.add, list(map(total, data)))

sumPageViews

In [ ]:
# Número de sessoes por usuário     
sessionByUser = pd.DataFrame({'qttSessions' : df.groupby('visitId').agg('size')})
sessionByUser

### Numero de sessoes distintas por dia

In [ ]:
# Numero de sessoes distintas por dia
distinctSessionsByDate = pd.DataFrame({'count' : df.groupby(['date']).agg('size')}).reset_index()
distinctSessionsByDate

### Sessões diárias por tipo de browser

In [ ]:
# Sessões diárias por tipo de browser
data = df[['date','device']]

# Simplifica coluna
dailySessions = data.assign(browser=data['device'].map( lambda row: row['browser']))
dailySessions = dailySessions.groupby(['date'])
dailySessionsByBrowser = dailySessions.apply(lambda dateGroup: dateGroup.groupby('browser').size() )

dailySessionsByBrowser

### Média do tempo da sessao por data

In [ ]:
# Média do tempo da sessao por data
data = df[['date','totals']]

# Filtra pelas linhas cujo campo totals contêm 'timeOnSite'
data = data[data['totals'].apply(check,args=('timeOnSite',))]

# Simplifica coluna
sessionTime = data.assign(totals=data['totals'].map(lambda row: int(row['timeOnSite'])))

sessionTimeByDate = sessionTime.groupby(['date']).mean()

sessionTimeByDate